# Volatility of the Simulated Data-Generating Processes (DGPs)

This notebook loads simulated **return series** from `datasets/`, estimates daily **volatility** with three baseline models, generates **comparison plots**, and (optionally) exports a **LaTeX table** with GARCH(1,1) long-run statistics.


In [1]:
from pathlib import Path
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch.univariate import arch_model

# Local utilities
from utils.evaluation import dataframe_to_latex
from utils.plotting import plot_volatilities_percent

### What the notebook does

- Reads all `*_returns.csv` files in `datasets/`  
  (skips `constant_returns.csv`, `linear_returns.csv`).
  
- Estimates volatility with:
  - **MA(22):** rolling standard deviation (22-day window).  
  - **EWMA(0.94):** RiskMetrics exponential smoothing.  
  - **GARCH(1,1) normal:** fitted with `arch`; returns scaled ×100 for stability, vol rescaled back.
- Saves one plot per dataset in `volatility_dgps/`:  
  - **GARCH** = red solid, **EWMA** = blue dotted, **MA** = black dashed.  
  - y-axis in percent, boxed axes, white background.
- Optionally writes a LaTeX table `volatility_dgps/garch_longrun_stats.tex`, containing:  
  - Estimated GARCH parameters ($\omega, \alpha, \beta$)  
  - Persistence ($\alpha+\beta$)  
  - Long-run volatility  
    $$
    \sigma_\infty = \sqrt{\tfrac{\omega}{1-\alpha-\beta}}, \quad (\alpha+\beta < 1)
    $$  
  - Half-life of shocks  
    $$
    \text{HL} = \frac{\ln(0.5)}{\ln(\alpha+\beta)}
    $$
- **Inputs:** return series in CSV format from `datasets/` (one numeric column, header optional).  
- **Outputs:** per-DGP volatility plots (`volatility_dgps/<dgp_name>_vol.png`) and optional LaTeX table with GARCH(1,1) long-run stats.

In [2]:
# Paths and Params
datasets_folder = Path("datasets")
plots_folder = Path("volatility_dgps")
plots_folder.mkdir(exist_ok=True)

ma_window_length = 22
ewma_lambda_decay = 0.94

In [3]:
# Utils 
def vol_find_return_files(datasets_path: Path) -> list[Path]:
    my_paths = set(glob.glob(str(datasets_path / "*_returns.csv"))) | \
               set(glob.glob(str(datasets_path / "**/*_returns.csv"), recursive=True))
    my_files: list[Path] = []
    for my_path in sorted(map(Path, my_paths)):
        nm = my_path.name.lower()
        if nm in {"constant_returns.csv", "linear_returns.csv"}:
            continue
        my_files.append(my_path)
    return my_files


def vol_read_returns_series(csv_path: Path) -> pd.Series:
    try:
        df = pd.read_csv(csv_path, header=None)
    except Exception:
        df = pd.read_csv(csv_path)
    if df.shape[1] == 1:
        s = pd.to_numeric(df.iloc[:, 0], errors="coerce")
    else:
        cols = [c for c in df.columns if pd.to_numeric(df[c], errors="coerce").notna().any()]
        if not cols:
            raise ValueError(f"No numeric column found in {csv_path}")
        s = pd.to_numeric(df[cols[0]], errors="coerce")
    s = s.dropna().reset_index(drop=True).astype(float)
    s.name = "return"
    return s

In [4]:
# Volatility Models Functions
def compute_ma_volatility(returns_series: pd.Series,
                          window_length: int = 22,
                          ddof: int = 1) -> pd.Series:
    vol = returns_series.rolling(window=window_length, min_periods=window_length).std(ddof=ddof)
    vol.name = f"MA ({window_length}-day)"
    return vol


def compute_ewma_volatility(returns_series: pd.Series,
                            lambda_decay: float = 0.94,
                            init_variance: float | None = None) -> pd.Series:
    r = returns_series.values
    n = len(r)
    lam = float(lambda_decay)
    if not (0.0 < lam < 1.0):
        raise ValueError("lambda_decay must be in (0,1).")
    if init_variance is None:
        init_variance = float(np.var(r, ddof=1)) if n > 1 else float(np.var(r))

    sigma2 = np.empty(n, dtype=float)
    sigma2[0] = init_variance
    for i in range(1, n):
        sigma2[i] = lam * sigma2[i - 1] + (1.0 - lam) * (r[i - 1] ** 2)
    vol = pd.Series(np.sqrt(sigma2), index=returns_series.index, name=f"EWMA (λ = {lambda_decay})")
    return vol


def fit_garch11_volatility_and_stats(returns_series: pd.Series,
                                     scale: float = 100.0):
    """
    GARCH(1,1) normal, zero-mean. Scale returns by 100 for stability, then scale vol back.
    Returns (cond_vol_series, stats_dict).
    """
    scaled = returns_series.values * scale
    am = arch_model(y=scaled, mean="Zero", vol="GARCH", p=1, q=1, dist="normal", rescale=True)
    res = am.fit(disp="off")

    cond_vol = pd.Series(res.conditional_volatility / scale,
                         index=returns_series.index, name="GARCH (1,1)")

    params = res.params
    omega = float(params.get("omega", np.nan))
    alpha = float(params.get("alpha[1]", np.nan))
    beta  = float(params.get("beta[1]",  np.nan))
    alpha_plus_beta = alpha + beta

    long_run_var_scaled = np.nan
    if np.isfinite(alpha_plus_beta) and (1.0 - alpha_plus_beta) != 0:
        long_run_var_scaled = omega / (1.0 - alpha_plus_beta)
    long_run_var = long_run_var_scaled / (scale ** 2) if np.isfinite(long_run_var_scaled) else np.nan
    long_run_vol = np.sqrt(long_run_var) if (np.isfinite(long_run_var) and long_run_var >= 0) else np.nan

    if 0.0 < alpha_plus_beta < 1.0:
        half_life_days = np.log(0.5) / np.log(alpha_plus_beta)
    else:
        half_life_days = np.inf

    stats = {
        "omega": omega,
        "alpha": alpha,
        "beta": beta,
        "alpha_plus_beta": alpha_plus_beta,
        "long_run_vol": long_run_vol,
        "long_run_vol_pct": long_run_vol * 100.0 if np.isfinite(long_run_vol) else np.nan,
        "half_life_days": half_life_days
    }
    return cond_vol, stats

In [5]:
# Collect per-DGP stats and export a LaTeX table
def process_all_and_plot_with_table(datasets_path: Path,
                                    output_folder: Path,
                                    ma_window: int = ma_window_length,
                                    ewma_lambda: float = ewma_lambda_decay,
                                    latex_path: Path | None = None):
    files = vol_find_return_files(datasets_path)
    if not files:
        print("No eligible *_returns.csv files found.")
        return

    rows = []

    for f in files:
        dgp_key = f.stem.replace("_returns", "")
        print(f"[INFO] Plotting: {dgp_key}")
        r = vol_read_returns_series(f)

        vol_ma = compute_ma_volatility(r, window_length=ma_window)
        vol_ewma = compute_ewma_volatility(r, lambda_decay=ewma_lambda)
        vol_garch, stats = fit_garch11_volatility_and_stats(r)

        png = output_folder / f"{dgp_key}_vol.png"
        plot_volatilities_percent(vol_ma, vol_ewma, vol_garch,
                                  title_text=f"Volatility Comparison — {dgp_key}",
                                  save_path=png)

        rows.append({
            "model_name": "GARCH(1,1)-normal",
            "dgp_type": dgp_key,
            "context_length": len(r),
            "omega": stats["omega"],
            "alpha": stats["alpha"],
            "beta": stats["beta"],
            "alpha_plus_beta": stats["alpha_plus_beta"],
            "long_run_vol_%": stats["long_run_vol_pct"],
            "half_life_days": stats["half_life_days"]
        })

    if latex_path is not None and rows:
        df_stats = pd.DataFrame(rows).set_index(["model_name", "dgp_type", "context_length"])
        dataframe_to_latex(df_stats, output_path=latex_path, preserve_index_order=True)
        print(f"[DONE] LaTeX table saved to: {latex_path.resolve()}")

In [6]:
# Run 
latex_output_path = plots_folder / "garch_longrun_stats.tex"

process_all_and_plot_with_table(
    datasets_path=datasets_folder,
    output_folder=plots_folder,
    ma_window=ma_window_length,
    ewma_lambda=ewma_lambda_decay,
    latex_path=latex_output_path
)

[INFO] Plotting: gbm_high_vol
[INFO] Plotting: gbm_low_vol
[INFO] Plotting: mixture_normal
[INFO] Plotting: seasonal
[INFO] Plotting: t_garch
[DONE] LaTeX table saved to: /Users/aledo/GitHub/ts-transform-matherika-internal/simulations_new_setup/volatility_dgps/garch_longrun_stats.tex
